In [1]:
# -*- coding: utf-8 -*-

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parse
from time import sleep
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import random
import matplotlib.pyplot as plt
import math
% matplotlib inline

In [3]:
import lxml
import requests
import html5lib
pd.options.display.max_rows = 50

In [4]:
from bokeh.models import HoverTool, CategoricalColorMapper, OpenURL, TapTool, ResetTool
from bokeh.plotting import ColumnDataSource, figure, output_file, show, save, output_notebook
from bokeh.io import curdoc
from bokeh.models.glyphs import VBar

In [ ]:
table_pages = ['https://en.wikipedia.org/wiki/List_of_battles_before_301',
               'https://en.wikipedia.org/wiki/List_of_battles_301%E2%80%931300',
               'https://en.wikipedia.org/wiki/List_of_battles_1301%E2%80%931600',
               'https://en.wikipedia.org/wiki/List_of_battles_since_2001']

In [ ]:
list_pages = ['https://en.wikipedia.org/wiki/List_of_battles_1601%E2%80%931800',
               'https://en.wikipedia.org/wiki/List_of_battles_1801%E2%80%931900',
               'https://en.wikipedia.org/wiki/List_of_battles_1901%E2%80%932000',]

In [ ]:
def list_battles(url):
    req = requests.get(url)
    print req.status_code
    era = req.content
    era_soup = BeautifulSoup(era)
    bullets = era_soup.find_all('li')
    
    for bullet in bullets:
        print bullet
        if 'title' in str(bullet):
#             if 'expedition' or 'recapture' or 'capture' or 'war' or 'action' or 'battle' or 'siege' or 'operation' or 'fall' in str(bullet).lower():
#                 if 'List+of+battles' not in str(bullet):
                    if 'footer' not in str(bullet):
                        link = bullet.a['href']
                        link_list.append(link)

                        title = bullet.a['title']
                        title_list.append(title)
                    else:
                        print 'NEXT ENTRY: ' + str(bullet)
#                 else:
#                     print 'NEXT ENTRY: ' + str(bullet)
#             else:
#                 print 'NEXT ENTRY: ' + str(bullet)
        else: 
            print 'NEXT ENTRY' + str(bullet)
                    

In [ ]:
title_list = []
link_list = []
for url in list_pages:
    list_battles(url)

In [ ]:
link_list

In [ ]:
df_late_battles = pd.concat([pd.Series(link_list), pd.Series(title_list)], axis = 1).drop_duplicates()

In [ ]:
df_late_battles = df_late_battles.rename(columns = {0:'url', 1:'Battle'})

In [ ]:
df_late_battles

In [ ]:
df_late_battles[~df_late_battles['Battle'].str.contains('Battle' or 'Operation')]

In [ ]:
df_late_battles.to_csv('battle_list_1600-', encoding = 'utf-8')

In [ ]:
df_late_battles = pd.read_csv('battle_list_1600-.csv', encoding = 'utf-8')

In [ ]:
df_late_battles.head()

In [ ]:
df_late_battles = df_late_battles.drop(['Column1', 'Column4'], axis = 1)
df_late_battles.columns = df_late_battles.iloc[0]
df_late_battles = df_late_battles.drop(0)

In [ ]:
battle_list = []
link_list = []

In [ ]:
def early_battles(link):
    req = requests.get(link)
    print req.status_code
    era = req.content
    era_soup = BeautifulSoup(era)
    tables = era_soup.find_all('table', {'class': 'wikitable'})

    for table in tables:
        cells = table.find_all('td')
        for cell in cells:
            if cell.a is not None:
                a = ['Fall', 'Battle', 'Siege', 'Capture', 'Operation', 'Action', 'Recapture']
                if any(x in str(cell.a['href']) for x in a):
                    battle_list.append(cell.a['title'])
                    link_list.append(cell.a['href'])

In [ ]:
for page in table_pages:
    print page
    early_battles(page)

In [ ]:
df_battles = pd.DataFrame({'battle': battle_list,
                          'link':link_list})

In [ ]:
df_battles = df_battles.rename(columns = {'link':'url', 'battle':'Battle'})

In [ ]:
len(df_battles)

In [ ]:
len(df_late_battles)

In [ ]:
df_battles.head()

In [ ]:
df_url = pd.concat([df_late_battles, df_battles]).reset_index(drop = True)

In [ ]:
len(df_url)

In [ ]:
len(df_url.drop_duplicates())

In [ ]:
s_req = requests.get('https://en.wikipedia.org/Alexander_the_Great')

soup = BeautifulSoup(s_req.text)

ul = soup.find('ul', {'class':'redirectText'})

In [ ]:
if ul is not None:
    print 'works'

In [ ]:
df_battle_all = pd.DataFrame()
df_battle = pd.DataFrame()
def table_scrape(url, name):
#     sleep(1)
    print name
    link = 'https://en.wikipedia.org' + url
    b_req = requests.get(link)

    soup = BeautifulSoup(b_req.text)
    
    bell = soup.find('table', {'class':'infobox vevent'})
    
    if bell is not None:

        details = pd.DataFrame(parse.make2d(bell))
    else:
        return b_req
    

    if 'Belligerents' in str(bell):
    
        clean_rows = details.drop_duplicates(0).set_index(0,drop = True).loc[['Date', 'Location', 'Result'],:]

        clean_col = clean_rows.transpose().reset_index(drop = True)

        df_x_bells = pd.DataFrame()

        html_table = bell.find_all('tr')
        bell_list = []
        bell_y_list = []

        for index, cell in zip(range(0, len(html_table)), html_table):
            if 'Commanders and leaders' in str(cell):
                leaders = html_table[index + 1]
                cells = leaders.find_all('td')
                count = 0
                for cell in cells:
                    count += 1
                    anchors = cell.find_all('a')
                    for anchor in anchors:
                        if 'title' in str(anchor):
                            if 'class="image"' in str(anchor):
                                cl = 'image'
                            elif 'class="thumbborder"' in str(anchor):
                                cl = 'thumb'
                            elif 'cite_note' in str(anchor):
                                cl = 'cite'
                            elif 'disambiguation needed' in str(anchor):
                                cl = 'disam'
                            elif 'cnote_g' in str(anchor):
                                cl = 'cnote'
                            else:
                                title = anchor['title']
                                if any(title == x for x in ['Napoleon I','Alexander III of Macedon']):
                                    print 'napoleon or alex redirect'

                                ref = anchor['href']
                                link = 'https://en.wikipedia.org' + ref

                                if len(ref) <= 10:
                                    t_req = requests.get(link)
                                elif any(ref[8] + ref[9] + ref[10] == x for x in ['pt.', 'tr.', 'ko.', 'ja.', 'th.','da.', 'es.', 'de.', 'it.', 'fr.', 'zh.']):
                                    print 'Not English!'
                                    link = ref
                                    t_req = requests.get(link)
                                elif soup.find('ul', {'class':'redirectText'}) is not None:
                                    print 'REDIRECT'
                                    print title
                                    ul = soup.find('ul', {'class':'redirectText'})
                                    a = ul.a
                                    redirect = a['href']
                                    link = 'https://en.wikipedia.org' + redirect
                                    t_req = requests.get(link)
                                else:
                                    t_req = requests.get(link)

                                t_soup = BeautifulSoup(t_req.text)

                                c_name = t_soup.find('title')
                                c_name = str(c_name)

                                if c_name:
                                    c_name = c_name.replace(' - Wikipedia', '').replace('<title>', '').replace('</title>', '')
                                else:
                                    if count == 1:
                                        bell_list.append(title)
                                    elif count == 2:
                                        bell_y_list.append(title)
                                if 'ex.php?title=' in ref:
                                    ref = ref.replace('ex.php?title=', '')
                                    ref = ref.replace('&action=edit&redlink=1', '')

                                omit_list = ['Capital punishment', 
                                             'Military advisor', 
                                             'Wounded in action', 
                                             'Prisoner of war', 
                                             'Killed in action', 
                                             'Surrender (military)', 
                                             'Surrendered',
                                             'Common military ranks in English']

                                if all(c_name != x for x in omit_list):


                                    if count == 1:

                                        bell_list.append(c_name)
                                    elif count == 2:
                                        bell_y_list.append(c_name)



        for bell in bell_list:
            clean_col['belligerent'] = bell
            df_x_bells = pd.concat([df_x_bells, clean_col])
            df_x_bells = df_x_bells.reset_index(drop = True)


        df_y_bells = pd.DataFrame()
        for bell in bell_y_list:
            clean_col['belligerent'] = bell
            df_y_bells = pd.concat([df_y_bells, clean_col])
            df_y_bells = df_y_bells.reset_index(drop = True)

        df_y_bells = df_y_bells.rename(columns = {0:'Location'})

        if 'Strength' in str(details[0]):
            strength_finder = details[details[0] == 'Strength']


            strength_row = strength_finder.index + 1

            strength = details.loc[strength_row]

            strength_x = strength.iloc[0,0]

            strength_y = strength.iloc[0,1]
        else:
            strength_x = 'unknown'
            strength_y = 'unknown'

        df_x_bells['own'] = strength_x

        df_x_bells['opp'] = strength_y

        df_y_bells['own'] = strength_y

        df_y_bells['opp'] = strength_x

        if 'Casualties and losses' not in str(details[0]):
            df_x_bells['taken'] = np.NaN
            df_x_bells['inflicted'] = np.NaN
            df_y_bells['taken'] = np.NaN
            df_y_bells['inflicted'] = np.NaN

        else:

            casualties_finder = details[details[0] == 'Casualties and losses']

            casualties_row = casualties_finder.index + 1

            casualties = details.loc[casualties_row]

            casualties_x = casualties.iloc[0,0]

            casualties_y = casualties.iloc[0,1]

            df_x_bells['taken'] = casualties_x

            df_x_bells['inflicted'] = casualties_y

            df_y_bells['taken'] = casualties_y

            df_y_bells['inflicted'] = casualties_x



        df_x_bells['Battle'] = name

        df_y_bells['Battle'] = name

        df_x_bells['pos'] = 'L'

        df_y_bells['pos'] = 'R'

        df_battle = pd.concat([df_x_bells, df_y_bells]).reset_index(drop = True)

        df_battle_all = pd.concat([df_battle_all, df_battle]).reset_index(drop = True).drop_duplicates()

        global df_battle_all
    else:
        print 'Filtered: ' + name

In [ ]:
count = 0
for num, rows in {1:df_url[:200],
             2:df_url[200:400], 
             3:df_url[400:600], 
             4:df_url[600:800],
             5:df_url[800:1000],
             6:df_url[1000:1200],
             7:df_url[1200:1400],
             8:df_url[1400:1600],
             9:df_url[1600:1800],
             10:df_url[1800:2000],
             11:df_url[2000:2200],
             12:df_url[2200:2400],
             13:df_url[2400:2600],
             14:df_url[2600:2800],
             15:df_url[2800:3000],
             16:df_url[3000:3200],
             17:df_url[3200:3400],
             18:df_url[3400:]}.iteritems():
    df_battle = pd.DataFrame()
    df_batch = rows
    for index, row in df_batch.iterrows():
        count += 1
        global count
        print str(count) + '/' + str(len(df_url))
        table_scrape(row['url'], row['Battle'])
    df_battle_all.to_csv('rows_{}.csv'.format(num), encoding = 'utf-8')
    print 'NEW FRAME' + str(num)

In [ ]:
row_list = ['rows_1.csv',
 'rows_2.csv',
 'rows_3.csv',
 'rows_4.csv',
 'rows_5.csv',
 'rows_6.csv',
 'rows_7.csv',
 'rows_8.csv',
 'rows_9.csv',
 'rows_10.csv',
 'rows_11.csv',
 'rows_12.csv',
 'rows_13.csv',
 'rows_14.csv',
 'rows_15.csv',
 'rows_16.csv',
 'rows_17.csv',
 'rows_18.csv']

In [ ]:
df_comm = pd.DataFrame()
for sheet in row_list:
    df_rows = pd.read_csv(sheet)
    df_comm = pd.concat([df_comm, df_rows]).reset_index(drop = True)

In [ ]:
df_comm = df_comm.drop('Unnamed: 0', axis = 1)

In [ ]:
len(df_comm)

In [ ]:
len(df_comm)

In [ ]:
df_comm.to_csv('battles_dirty.csv', encoding = 'utf-8')

In [ ]:
df_comm = pd.read_csv('battles_dirty.csv').drop('Unnamed: 0', axis = 1)

In [ ]:
df_comm = df_comm.drop_duplicates()

In [ ]:
df_comm.head()

In [ ]:
len(df_comm)

In [ ]:
df_comm[df_comm.belligerent.str.contains('Nasser')]

In [ ]:
len(df_comm.drop_duplicates())

In [ ]:
df_comm = df_comm.drop_duplicates().reset_index(drop = True)

In [ ]:
df_comm.to_csv('battles_deduped.csv')

In [ ]:
len(df_comm[df_comm.belligerent.str.contains('Napoleon')])

In [ ]:
df_comm[df_comm.belligerent.str.contains('Scipio')]

In [ ]:
df_results = pd.read_csv('result_column.csv')

In [ ]:
df_results[df_results.Battle == 'Battle_of_Camarón']

In [ ]:
df_comm.head()

In [ ]:
df_results.head()

In [ ]:
df_more_results = pd.merge(df_comm, df_results, how = 'outer', on = ['Battle', 'belligerent', 'pos', 'Result'])

In [ ]:
df_more_results = df_more_results[df_more_results.VorD.isnull()]

In [ ]:
df_more_results = df_more_results.drop_duplicates(subset = ['Battle', 'Date', 'VorD'])

In [ ]:
df_more_results.head()

In [ ]:
df_more_results.to_csv('adding_results_10.21.csv', encoding = 'utf-8')

In [ ]:
df_comm[df_comm.Battle == 'Battle_of_Camarón']

In [ ]:
df_comm[df_comm.belligerent == 'Ambroise de Loré']

In [ ]:
df_vl = pd.merge(df_comm, df_results, how = 'outer', on = ['Battle', 'belligerent', 'Result', 'pos'])

In [ ]:
df_vl

In [ ]:
df_vl = df_vl.sort('VorD')

In [ ]:
df_vl[df_vl.belligerent == 'Ambroise de Loré']

In [ ]:
df_vl[df_vl.belligerent == 'Saracens']

In [ ]:
df_vl.to_csv('additional_results.csv')

In [ ]:
df_10_21_fill = df_vl[df_vl.VorD.isnull()]

In [ ]:
df_ar = pd.read_csv('additional_results.csv', encoding = 'utf-8')

In [ ]:
df_strength = df_ar.drop(['Unnamed: 0', 'Location', 'Result', 'inflicted', 'taken', 'opp'], axis = 1)

In [ ]:
df_strength = df_strength.sort('Battle').reset_index(drop = True)

In [ ]:
df_strength_dd = df_strength.drop_duplicates(['own', 'pos'], keep = 'first')

In [ ]:
df_strength_dd.own = df_strength_dd.own.str.replace('\r',' ').str.replace('\n', ' ').replace('\n', ' ')

In [ ]:
df_strength_dd = df_strength_dd.sort('Battle').reset_index(drop = True)

In [ ]:
df_strength_dd

In [ ]:
df_strength_dd.to_csv('strength_entry.csv', encoding = 'utf-8')

In [ ]:
df_strength_new = df_strength.drop_duplicates(['own', 'pos', 'Battle'], keep = 'first')

In [ ]:
df_strength_fill = pd.read_excel('strength_entry.xlsx', encoding = 'utf-8')

In [ ]:
df_battles = pd.read_csv('battles_deduped.csv', encoding = 'utf-8').drop('Unnamed: 0', axis = 1)

In [ ]:
df_battles = df_battles.drop(['taken', 'inflicted'], axis = 1)

In [ ]:
df_strength_fill.Battle = df_strength_fill.Battle.str.replace(u"â€“",u'–')

In [ ]:
df_some_strength = pd.merge(df_battles, df_strength_fill, how = 'outer', on = ['Battle', 'pos']).drop(['own_y', 'belligerent_y', 'Date_y'], axis = 1).rename(columns = {'Date_x':'Date','own_x':'own', 'belligerent_x':'belligerent'})

In [ ]:
df_some_strength[df_some_strength.Battle.str.contains('Battle_of_Jena')]

In [ ]:
df_test = df_some_strength.iloc[:,9:]

In [ ]:
df_str_two = df_some_strength[df_some_strength['Infantry'].isnull() & df_some_strength['Cavalry'].isnull() & df_some_strength['Artillery'].isnull() & df_some_strength['Ships'].isnull() & df_some_strength['Airforce'].isnull() & df_some_strength['Special'].isnull()]

In [ ]:
df_str_two.head()

In [ ]:
df_str_two = df_str_two.drop_duplicates(['Battle', 'Date', 'Location', 'Result', 'pos']).drop('VorD', axis = 1)

In [ ]:
len(df_str_two)

In [ ]:
df_merger = df_strength.drop(['Date', 'belligerent', 'own'], axis = 1)

In [ ]:
df_merger = df_merger.drop_duplicates()

In [ ]:
df_str_two.head()

In [ ]:
df_str_two = pd.merge(df_str_two, df_merger, on = ['Battle', 'pos'], how = 'outer').dropna(subset = ['Date', 'Location', 'Result'])

In [ ]:
df_str_two = df_str_two.dropna(subset = ['VorD'])

In [ ]:
df_str_two.to_csv('strength_entry_two.csv')

In [ ]:
df_str_two

In [ ]:
df_partial = pd.read_excel('PARTIAL_strength_entry_two.xlsx', encoding = 'utf-8')

In [ ]:
df_str_two.head()

In [ ]:
df_str_two = df_str_two.drop(['Infantry', 'Cavalry', 'Artillery', 'Ships', 'Airforce', 'Special'], axis = 1)

In [ ]:
df_partial = df_partial.drop(['Date', 'Location', 'Result', 'belligerent','own', 'Column1'],axis = 1)

In [ ]:
df_partial.columns

In [ ]:
df_str_two.head()

In [ ]:
df_finish = pd.merge(df_str_two, df_partial, on = ['Battle', 'pos', 'VorD'], how = 'outer')

In [ ]:
drop_list = list(df_finish[df_finish['Date'].isnull()].index)

In [ ]:
df_finish = df_finish.drop(drop_list)

In [ ]:
df_finish.to_csv('last_strength_entry.csv', encoding = 'utf-8')

In [ ]:
df_some_strength

In [ ]:
df_some_strength[df_some_strength.Battle.str.contains('Siege of La Rochelle')]

In [ ]:
df_more_strength = pd.read_excel('last_strength_entry.xlsx')

In [ ]:
df_more_strength.head()

In [ ]:
df_some_strength.head()

In [ ]:
df_more_strength = pd.merge(df_more_strength, df_str_two, on = ['Battle', 'pos'], how = 'outer').drop(['own_y', 'Infantry_y', 'Cavalry_y', 'Artillery_y', 'Ships_y', 'Airforce_y', 'Special_y', 'Date_y', 'Location_y', 'Result_y', 'belligerent_y'], axis = 1).dropna(subset = ['Date_x'])

In [ ]:
df_more_strength.columns = df_more_strength.columns.str.replace('_x', '')

In [ ]:
df_more_strength.head()

In [ ]:
df_some_strength.head()

In [ ]:
df_more_strength

In [ ]:
df_fill_two = pd.merge(df_battles, df_more_strength, on = ['Battle', 'pos'], how = 'outer').drop(['Date_y', 
                                                                                    'Location_y', 
                                                                                    'belligerent_y', 
                                                                                    'Result_y', 'own_y', 
                                                                                    'opp_y'], axis = 1)

In [ ]:
df_fill_two.columns = df_fill_two.columns.str.replace('_x', '')

In [ ]:
df_some_strength = df_some_strength[df_some_strength['Infantry'].notnull() | df_some_strength['Cavalry'].notnull() | df_some_strength['Artillery'].notnull() | df_some_strength['Ships'].notnull() | df_some_strength['Airforce'].notnull() | df_some_strength['Special'].notnull()]

In [ ]:
len(df_some_strength)

In [ ]:
df_fill_two = df_fill_two[df_fill_two['Infantry'].notnull() | df_fill_two['Cavalry'].notnull() | df_fill_two['Artillery'].notnull() | df_fill_two['Ships'].notnull() | df_fill_two['Airforce'].notnull() | df_fill_two['Special'].notnull()]

In [ ]:
df_some_strength

In [ ]:
df_fill_two

In [ ]:
df_all_strength = pd.concat([df_fill_two, df_some_strength]).reset_index(drop = True)

In [ ]:
len(df_battles)

In [ ]:
df_char = df_all_strength[df_all_strength.Date.isnull()]

In [ ]:
df_char.to_excel('special_character_fix.xlsx')

In [ ]:
df_all_strength[df_all_strength.Battle.str.contains('Battle_of_Tenn')]

In [ ]:
df_all_strength.to_excel('all_strength_probably.xlsx')

In [ ]:
df_battles[df_battles.Battle.str.contains('dlingen')]

In [ ]:
df_char.Battle

In [5]:
df_strength_all = pd.read_excel('all_strength_probably.xlsx')

In [6]:
df_strength_all.loc[9988, 'Infantry'] = 2000

In [7]:
df_strength_all = df_strength_all.drop_duplicates()

In [8]:
df_left = df_strength_all[df_strength_all.pos == 'L']

In [9]:
df_left = df_left.drop(['Date', 'Location', 'Result', 'belligerent', 'opp', 'own'], axis = 1)

In [10]:
df_left = df_left.drop_duplicates(subset = ['Battle'])

In [11]:
df_left = df_left.dropna(subset = ['Infantry', 'Cavalry', 'Artillery', 'Ships', 'Airforce', 'Special'], how = 'all')

In [12]:
df_right = df_strength_all[df_strength_all.pos == 'R']

In [13]:
df_right = df_right.drop_duplicates(subset = ['Battle'])

In [14]:
df_right = df_right.drop(['Date', 'Location', 'Result', 'belligerent', 'opp', 'own', 'VorD'], axis = 1)

In [15]:
df_right = df_right.dropna(subset = ['Infantry', 'Cavalry', 'Artillery', 'Ships', 'Airforce', 'Special'], how = 'all')

In [16]:
df_model = pd.merge(df_left, df_right, on = 'Battle', how = 'outer').dropna(subset = ['pos_x', 'pos_y']).reset_index(drop = True)

In [17]:
df_v = df_model[df_model.VorD == 'V']

In [18]:
v_list = list(df_v.index)

In [19]:
ran_list = random.sample(v_list, 374)

In [20]:
df_switch = df_model.iloc[ran_list]

In [21]:
df_switch['Infantry_hold'] = df_switch['Infantry_x']
df_switch['Cavalry_hold'] = df_switch['Cavalry_x']
df_switch['Artillery_hold'] = df_switch['Artillery_x']
df_switch['Ships_hold'] = df_switch['Ships_x']
df_switch['Airforce_hold'] = df_switch['Airforce_x']
df_switch['Special_hold'] = df_switch['Special_x']

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [22]:
df_switch['Infantry_x'] = df_switch['Infantry_y']
df_switch['Cavalry_x'] = df_switch['Cavalry_y']
df_switch['Artillery_x'] = df_switch['Artillery_y']
df_switch['Ships_x'] = df_switch['Ships_y']
df_switch['Airforce_x'] = df_switch['Airforce_y']
df_switch['Special_x'] = df_switch['Special_y']

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [23]:
df_switch['Infantry_y'] = df_switch['Infantry_hold']
df_switch['Cavalry_y'] = df_switch['Cavalry_hold']
df_switch['Artillery_y'] = df_switch['Artillery_hold']
df_switch['Ships_y'] = df_switch['Ships_hold']
df_switch['Airforce_y'] = df_switch['Airforce_hold']
df_switch['Special_y'] = df_switch['Special_hold']

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [24]:
df_switch = df_switch.drop(['Infantry_hold', 'Cavalry_hold', 'Artillery_hold', 'Ships_hold', 'Airforce_hold', 'Special_hold'], axis = 1)

In [25]:
df_switch.VorD = 'D'

In [26]:
df_model = df_model.drop(ran_list)

In [27]:
df_model = pd.concat([df_model, df_switch])

In [28]:
df_model.loc[2378, 'Infantry_y'] = 25000

In [29]:
df_model.loc[2363, 'Infantry_x'] = 500

In [30]:
df_model = df_model[df_model.Infantry_x != 1.0]
df_model = df_model[df_model.Infantry_y != 1.0]

In [31]:
df_model = df_model.fillna(0)

In [32]:
df_model['infantry_diff'] = (df_model['Infantry_x'] - df_model['Infantry_y']) / (df_model['Infantry_x'] + df_model['Infantry_y'])
df_model['cavalry_diff'] = (df_model['Cavalry_x'] - df_model['Cavalry_y']) / (df_model['Cavalry_x'] + df_model['Cavalry_y'])
df_model['artillery_diff'] = (df_model['Artillery_x'] - df_model['Artillery_y']) / (df_model['Artillery_x'] + df_model['Artillery_y'])
df_model['ships_diff'] = (df_model['Ships_x'] - df_model['Ships_y']) / (df_model['Ships_x'] + df_model['Ships_y'])
df_model['airforce_diff'] = (df_model['Airforce_x'] - df_model['Airforce_y']) / (df_model['Airforce_x'] + df_model['Airforce_y'])
df_model['special_diff'] = (df_model['Special_x'] - df_model['Special_y']) / (df_model['Special_x'] + df_model['Special_y'])

In [33]:
df_diff = pd.concat([df_model['VorD'], df_model['Battle'], df_model['infantry_diff'],df_model['cavalry_diff'],df_model['artillery_diff'], df_model['ships_diff'],
                    df_model['airforce_diff'], df_model['special_diff']], axis = 1)

In [34]:
df_diff = df_diff[df_diff.VorD != 'I']

In [35]:
df_diff = df_diff.drop(174)

In [36]:
df_diff = df_diff.fillna(0)

In [37]:
df_x = pd.DataFrame(df_diff.iloc[:, 2:])

In [38]:
df_x = df_x.drop('artillery_diff', axis = 1)

In [39]:
df_y = pd.DataFrame(df_diff.iloc[:, 0])

In [40]:
df_y.VorD = df_y.VorD.replace('V', 1).replace('D', 0).replace('L', 0).replace('v', 1)

In [41]:
df_y.VorD = df_y.VorD.astype('float')

In [42]:
df_x = df_x.iloc[:, :-1]

In [43]:
lr = LogisticRegression()
lr.fit(df_x, df_y)

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [44]:
lr.score(df_x, df_y)

0.54402834008097167

In [45]:
df_strength_all.loc[60, 'VorD'] = 'V'
df_strength_all.loc[61, 'VorD'] = 'V'

In [46]:
df_strength_all.loc[3479:3485, 'VorD'] = 'V'

In [47]:
df_strength_all.loc[3485, 'VorD'] = 'D'

In [48]:
df_strength_all.loc[3802:3803, 'VorD'] = 'V'
df_strength_all.loc[3804:3807, 'VorD'] = 'D'

In [49]:
df_strength_all.loc[4142:4143, 'VorD'] = 'D'

In [50]:
df_strength_all.VorD = df_strength_all.VorD.str.replace('L', 'D')

In [51]:
df_strength_all.VorD = df_strength_all.VorD.str.replace('v', 'V')

In [52]:
df_strength_all.VorD = df_strength_all.VorD.str.replace('W', 'V')

In [53]:
null_fill = df_strength_all[df_strength_all.VorD.isnull()]

In [54]:
null_fill['VorD'] = ['V', 'D', 'D', 'V', 'V', 'D', 'V', 'V', 'V', 'V', 'D', 'D', 'D', 'D', 'V', 'V', 'V', 'V', 'V',
                    'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D','D', 'D', 'D', 'D', 'V', 'V', 'V', 'V', 'V', 'D', 'D', 'D', 'D',
                    'V', 'V', 'D', 'D', 'D', 'D', np.NaN, 'D', np.NaN]

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [55]:
df_strength_all.VorD.loc[null_fill.index] = null_fill['VorD']

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [56]:
df_strength_all = df_strength_all[df_strength_all.belligerent != 'Sweden']

In [57]:
import re

In [58]:
year_list = []
for i, row in df_strength_all.iterrows():
    d = row['Date']
    if isinstance(d, str):
        if re.search(r'[12]\d{3}', d):
            year = re.search(r'[12]\d{3}', d).group(0)
            year = int(year)
            if 'BC' in d:
                year = year * -1
            year_list.append({'Year':year, 'Index':i})
        elif re.search(r'[12]\d{2}', d):
            year = re.search(r'[12]\d{2}', d).group(0)
            year = int(year)
            if 'BC' in d:
                year = year * -1
            year_list.append({'Year':year, 'Index':i})
    

In [59]:
df_years = pd.DataFrame(year_list).set_index('Index', drop = True)

In [60]:
df_strength_all['year'] = pd.Series()

In [61]:
df_strength_all['year'].loc[df_years.index] = df_years.Year

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [62]:
df_strength_all.to_csv('current_run.csv', encoding = 'utf-8')

In [63]:
df_strength_all = pd.read_csv('current_run.csv', encoding = 'utf-8').drop('Unnamed: 0', axis = 1)

In [64]:
df_run = df_strength_all[df_strength_all.VorD.notnull()]

In [65]:
df_run.Battle = df_run.Battle.str.replace('_'," ")

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [66]:
arras_1 = {'Battle':['Battle of Arras', 'Battle of Arras'], 
           'Date':['21 May 1940', '21 May 1940'],
           'Location':['Arras', 'Arras'], 
           'Result':['German victory', 'German victory'] ,
           'belligerent':['Erwin Rommel', 'Harold Franklyn'],
           'opp':['manual', 'manual'],
           'own':['manual', 'manual'], 
           'pos':['R', 'L'],
           'VorD':['V', 'D'],
           'Infantry':[7500.0, 2000.0],
           'Cavalry':[225.0, 74.0],
           'Artillery':[np.NaN, np.NaN],
           'Ships': [np.NaN, np.NaN],
           'Airforce': [np.NaN, np.NaN],
           'Special': [np.NaN, np.NaN],
           'year':[1940, 1940]}

In [67]:
df_run = pd.concat([df_run,pd.DataFrame(arras_1, index = [10515,10516])])

In [68]:
df_run

,Airforce,Artillery,Battle,Cavalry,Date,Infantry,Location,Result,Ships,Special,VorD,belligerent,opp,own,pos,year
0,NaN,3.0,1898 invasion of GuantÃ¡namo Bay,NaN,NaN,5000.0,NaN,NaN,2.0,NaN,D,NaN,NaN,NaN,R,NaN
1,NaN,7.0,1898 invasion of GuantÃ¡namo Bay,NaN,NaN,5000.0,NaN,NaN,2.0,NaN,D,NaN,NaN,NaN,R,NaN
2,NaN,NaN,1898 invasion of GuantÃ¡namo Bay,NaN,NaN,923.0,NaN,NaN,6.0,NaN,V,NaN,NaN,NaN,L,NaN
3,NaN,NaN,1948 Arab–Israeli War,NaN,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...",117500.0,"Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,NaN,NaN,V,David Ben-Gurion,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,1948.0
4,NaN,NaN,1948 Arab–Israeli War,NaN,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...",117500.0,"Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,NaN,NaN,V,Yisrael Galili,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,1948.0
5,NaN,NaN,1948 Arab–Israeli War,NaN,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...",117500.0,"Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,NaN,NaN,V,Yaakov Dori,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,1948.0
6,NaN,NaN,1948 Arab–Israeli War,NaN,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...",117500.0,"Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,NaN,NaN,V,Yigael Yadin,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,1948.0
7,NaN,NaN,1948 Arab–Israeli War,NaN,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...",117500.0,"Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,NaN,NaN,V,Mickey Marcus,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,1948.0
8,NaN,NaN,1948 Arab–Israeli War,NaN,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...",117500.0,"Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,NaN,NaN,V,Yigal Allon,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,1948.0
9,NaN,NaN,1948 Arab–Israeli War,NaN,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...",117500.0,"Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,NaN,NaN,V,Yitzhak Rabin,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,1948.0


In [69]:
df_lookup = df_strength_all.drop_duplicates(subset = ['Battle', 'pos'])

In [70]:
df_lookup.Battle = df_lookup.Battle.str.replace('_', ' ')

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [71]:
df_lookup = pd.concat([df_lookup, pd.DataFrame(arras_1, index = [10515,10516])])

In [ ]:
war_list = []
battle_list = []
def commander_war(general):
#     print general
    war = 0
    global df_battles
    df_dbg = df_run[df_run.belligerent == general]
    df_battles = pd.DataFrame()
    for battle in df_dbg.Battle.unique():


        row = df_dbg[df_dbg.Battle == battle].reset_index(drop = True)
        
        battle = row.loc[0,'Battle']

        df_str = df_lookup[df_lookup.Battle == battle].reset_index(drop = True)
        
        outcome = pd.DataFrame(row.loc[:,'VorD']).iloc[0,0]
        
        
        if row.loc[0, 'pos'] == 'L':
            opp_row = df_str[df_str.pos == 'R']
        if row.loc[0, 'pos'] == 'R':
            opp_row = df_str[df_str.pos == 'L']
            
       
            
        year = row.loc[0, 'year']

        own = row.drop(['Battle', 'Date', 'Location', 'Result', 'belligerent', 'opp', 'own', 'pos', 'VorD', 'year'], axis = 1)

        opp = opp_row.drop(['Battle', 'Date', 'Location', 'Result', 'belligerent', 'opp', 'own', 'pos', 'VorD', 'year'], axis = 1)
        
        opp = opp.rename(columns = {'Infantry':'infantry_opp', 
                                    'Cavalry':'cavalry_opp', 
                                    'Artillery':'artillery_opp', 
                                    'Ships':'ships_opp', 
                                    'Airforce':'airforce_opp',
                                    'Special':'special_opp'})

        opp = opp.reset_index(drop= True)

        pred_row = pd.concat([own, opp], axis = 1)
        
        pred_diff = pd.DataFrame()
        
        pred_diff['infantry_diff'] = (pred_row['Infantry'] - pred_row['infantry_opp']) / (pred_row['Infantry'] + pred_row['infantry_opp'])
        pred_diff['cavalry_dfff'] = (pred_row['Cavalry'] - pred_row['cavalry_opp']) / (pred_row['Cavalry'] + pred_row['cavalry_opp'])
        pred_diff['artillery_diff'] = (pred_row['Artillery'] - pred_row['artillery_opp']) / (pred_row['Artillery'] + pred_row['artillery_opp'])
        pred_diff['ships_diff'] = (pred_row['Ships'] - pred_row['ships_opp']) / (pred_row['Ships'] + pred_row['ships_opp'])
        pred_diff['airforce_diff'] = (pred_row['Airforce'] - pred_row['airforce_opp']) / (pred_row['Airforce'] + pred_row['airforce_opp'])
        pred_diff['special_diff'] = (pred_row['Special'] - pred_row['special_opp']) / (pred_row['Special'] + pred_row['special_opp'])

#         pred_diff = pred_diff.drop(['Infantry', 'Cavalry', 'Artillery', 'Ships', 'Airforce', 'Special', 'infantry_opp', 'cavalry_opp', 'artillery_opp', 'ships_opp', 'airforce_opp', 'special_opp'], axis = 1)

        pred_diff = pred_diff.fillna(0)

        pred_diff = pred_diff.drop(['artillery_diff', 'special_diff'], axis = 1)
        if battle == 'Siege of Kojinyama':
            print(pred_diff)
            
        df_pred = pd.DataFrame(lr.predict_proba(pred_diff))
        
#         if row.loc[0, 'Infantry'] == 1.0:
        

        if len(df_str) == 1:
            if row.loc[0, 'VorD'] == 'V':
                value = 0.5
#                 print 'UNKNOWN'
            elif row.loc[0, 'VorD'] == 'D':
                value = -.5
#                 print 'UNKNOWN'
            elif row.loc[0, 'VorD'] == 'I':
                value = 0
#                 print 'UNKNOWN'
        else:
            if df_str.Infantry.loc[0] == 1.0 or df_str.Infantry.loc[1] == 1.0:
                if row.loc[0, 'VorD'] == 'V':
                    value = 0.5
#                     print 'UNKNOWN'
                elif row.loc[0, 'VorD'] == 'D':
                    value = -.5
#                     print 'UNKNOWN'
                elif row.loc[0, 'VorD'] == 'I':
                    value = 0
#                     print 'UNKNOWN'
            else:
                if row.loc[0, 'VorD'] == 'V':
                    value = df_pred.iloc[0,0]
                elif row.loc[0, 'VorD'] == 'D':
                    value = 0 - df_pred.iloc[0,1]
                elif row.loc[0, 'VorD'] == 'I':
                    value = .5 - df_pred.iloc[0,1]
        
        

#         print u' '.join(battle).encode('utf-8').strip(' ') +': ' + str(value).encode('utf-8')

        
        
        
        war = war + value
        battle_list.append({'General':general,
                            'Battle': battle,
                            'Value': value,
                            'Year': year,
                            'Outcome': outcome})
        df_battles = pd.DataFrame(battle_list)
        
#     print 'Total: ' + str(war)
#     print u' '.join(general).encode('utf-8').strip(' ') + ': ' + str(war)
#     if len(df_dbg) > 4:
#         war_list.append({'General':general,
#                      'WAR':war})
    war_list.append({'General': general,
                    'WAR': war,
                    'Battles': len(df_dbg)})
    print(war)
    

In [ ]:
war_list = []
count = 0
for general in df_run.belligerent.unique():
    count += 1
    print(str(general) + ': ' + str(count) + '/' + str(len(df_run.belligerent.unique())))
    commander_war(general)

nan: 1/6620
0
David Ben-Gurion: 2/6620
0.982021143865
Yisrael Galili: 3/6620
0.469494434062
Yaakov Dori: 4/6620
0.469494434062
Yigael Yadin: 5/6620
0.469494434062
Mickey Marcus: 6/6620
0.469494434062
Yigal Allon: 7/6620
0.469494434062
Yitzhak Rabin: 8/6620
1.0369304262
David Shaltiel: 9/6620
0.469494434062
Moshe Dayan: 10/6620
2.10759744265
Shimon Avidan: 11/6620
0.469494434062
Moshe Carmel: 12/6620
0.469494434062
Yitzhak Sadeh: 13/6620
0.469494434062
Abdul Rahman Hassan Azzam: 14/6620
-0.461668151766
Farouk of Egypt: 15/6620
-0.461668151766
Abdullah I of Jordan: 16/6620
-0.461668151766
Muzahim al-Pachachi: 17/6620
-0.461668151766
Husni al-Za'im: 18/6620
-0.461668151766
Amin al-Husseini: 19/6620
-0.461668151766
Ahmed Ali al-Mwawi: 20/6620
-0.461668151766
Mohammed Naguib: 21/6620
-0.461668151766
John Bagot Glubb: 22/6620
-0.461668151766
Habis al-Majali: 23/6620
-0.461668151766
Hasan Salama: 24/6620
-0.461668151766
Arab Liberation Army: 25/6620
-0.461668151766
Fawzi al-Qawuqji: 26/6620
-

-0.5
Antigonus II Gonatas: 209/6620
0.5
Oda Nobunaga: 210/6620
4.27375868476
Tokugawa Ieyasu: 211/6620
1.3746279971
Inaba Yoshimichi: 212/6620
0.481430935317
Azai Nagamasa: 213/6620
-0.837854572643
Asakura Yoshikage: 214/6620
-1.36098892897
Timur: 215/6620
0.478634232314
Shahrukh Mirza: 216/6620
0.478634232314
Khalil Sultan: 217/6620
0.478634232314
Miran Shah: 218/6620
0.478634232314
Abu Bakr (Timurid): 219/6620
0.478634232314
Sultan Huseyn: 220/6620
0.478634232314
Pir Muhammad ibn Jahangir: 221/6620
0.478634232314
Taj al-Din Shah-i Shahan Abu'l Fath: 222/6620
0.478634232314
Bayezid I: 223/6620
-0.00493369529001
Süleyman Çelebi: 224/6620
-0.47079080044
Mehmed I: 225/6620
-0.47079080044
Stefan Lazarević: 226/6620
0.0148725970642
Gannys: 227/6620
0.5
Heraclius: 228/6620
-0.66436607343
Shahrbaraz: 229/6620
0.427905629374
Shahin Vahmanzadegan: 230/6620
0.5
Eslamu: 231/6620
-0.579003425802
Fatagar: 232/6620
-0.579003425802
Theophilos (emperor): 233/6620
-0.509572653682
Theophobos: 234/6620


Shalva of Akhaltsikhe: 410/6620
0.536564421481
Nouri al-Maliki: 411/6620
-0.0275534520095
Ali Ghaidan Majid: 412/6620
-0.0275534520095
Mohan al-Furayji: 413/6620
-0.0275534520095
Fadhil Jalil al-Barwari: 414/6620
-0.0275534520095
Muqtada al-Sadr: 415/6620
0.535385846498
Shiek Ali al-Sauidi: 416/6620
0.0353858464979
Akram al-Kabi: 417/6620
0.0353858464979
Yusuf al-Mosawi: 418/6620
0.0353858464979
John Stewart, Earl of Buchan: 419/6620
-0.604122090881
Gilbert Motier de La Fayette: 420/6620
0.44836241408
List of Marshals of France: 421/6620
0.44836241408
Thomas of Lancaster, 1st Duke of Clarence: 422/6620
-0.440595832954
Thomas Beaufort, Duke of Exeter: 423/6620
-0.440595832954
John Beaufort, 1st Duke of Somerset: 424/6620
-0.440595832954
Zhang Fei: 425/6620
1.05547430171
Zhang He: 426/6620
-0.0172028394698
Baldwin IV of Jerusalem: 427/6620
0.0520627780476
Baldwin of Ibelin: 428/6620
0
Balian of Ibelin: 429/6620
-0.96648111741
Hugh II of Saint Omer: 430/6620
0
Gökböri: 431/6620
0.89487477

Hanno, son of Bomilcar: 604/6620
0.532972404529
Hasdrubal (quartermaster): 605/6620
0.532972404529
Marcus Claudius Marcellus: 606/6620
0.492427596669
Lucius Manlius Vulso Longus: 607/6620
0.50816676581
Hanno the Great: 608/6620
-1.00030378335
Appius Claudius Pulcher (consul 212 BC): 609/6620
-0.526105410951
Francesco I Sforza: 610/6620
1.44741640467
Micheletto Attendolo: 611/6620
0.0418010160839
William Kirkcaldy of Grange: 612/6620
1.02855240568
James Hepburn, 4th Earl of Bothwell: 613/6620
-0.496068226645
Necho II: 614/6620
-0.535284530115
Nebuchadnezzar II: 615/6620
2.04309975946
Marcus Licinius Crassus: 616/6620
-0.485478535097
Publius Licinius Crassus (son of triumvir): 617/6620
-0.485478535097
Gaius Cassius Longinus: 618/6620
-1.45605196121
Surena: 619/6620
0.493338553091
Hasan ibn al-Nu'man: 620/6620
0.5
Tiberios III: 621/6620
-0.5
Hasdrubal the Boetharch: 622/6620
-0.5
Scipio Aemilianus: 623/6620
1.0
Manius Manilius: 624/6620
0.5
Alberico da Barbiano: 625/6620
0.5
Muzio Sforza:

0.498726615539
Theramenes: 807/6620
0.498726615539
Hermocrates: 808/6620
0.00913607904771
Gar Trinring Tsendro: 809/6620
0.423991743743
Xue Rengui: 810/6620
-0.41632871035
Ashina Daozi: 811/6620
-0.41632871035
Guo Daifeng: 812/6620
-0.41632871035
Abdul Qayyum Zakir: 813/6620
-0.5
Sancho d'Avila: 814/6620
1.03206133176
Mas'ud I of Ghazni: 815/6620
-0.549248554258
Ali Daya: 816/6620
-0.549248554258
Ahmad Shirazi: 817/6620
-0.549248554258
Abu Sahl Zawzani: 818/6620
-0.549248554258
Abd al-Razzaq Maymandi: 819/6620
-0.549248554258
Begtoghdi: 820/6620
-0.549248554258
Subashi (general): 821/6620
-0.549248554258
Chaghri Beg: 822/6620
0.557023286967
Tughril: 823/6620
0.557023286967
Faramurz: 824/6620
0.557023286967
Minamoto no Yoshitsune: 825/6620
1.56246994341
Taira no Munemori: 826/6620
-1.02484999159
Taira no Tomomori: 827/6620
-0.0248499915881
Perozes: 828/6620
-0.5305445743
Pityaxes: 829/6620
-0.5305445743
Baresmanas: 830/6620
-0.5305445743
Pharas the Herulian: 831/6620
0.538370772172
Tadd

...

David McMurtrie Gregg: 3824/6620
0.0451083732709
Friedrich-Georg Eberhardt: 3825/6620
0.395585884167
Józef Unrug: 3826/6620
-0.388090341486
Włodzimierz Steyer: 3827/6620
-0.388090341486
Adam Mohuczy: 3828/6620
-0.388090341486
Theophilus H. Holmes: 3829/6620
-0.526968706907
Benjamin Prentiss: 3830/6620
1.13139595184
Wilhelm von Tegetthoff: 3831/6620
-0.0180081573061
Edouard Suenson: 3832/6620
0.529782439485
Reginald Tyrwhitt: 3833/6620
0.477044721306
Roger Keyes, 1st Baron Keyes: 3834/6620
0.477044721306
Leberecht Maass: 3835/6620
-0.469203895445
Jørgen Rantzau: 3836/6620
-0.501168988319
Theodore Read: 3837/6620
0.0198511881608
Charles Edward Hovey: 3838/6620
0.604279180333
William Plummer Benton: 3839/6620
0.604279180333
Mnyamana Buthelezi: 3840/6620
0.406930473558
Evelyn Wood (British Army officer): 3841/6620
0.187659249457
Liu Yongfu: 3842/6620
-1.11303270085
Laurent Giovanninelli: 3843/6620
0.532525558671
General of the branch: 3844/6620
0.460944796465
Franz Moritz von Lacy: 3845/66

0.455204879214
Rathvon M. Tompkins: 4005/6620
-0.0104840623728
David E. Lownds: 4006/6620
-0.0104840623728
Trần Quý Hai: 4007/6620
0.0183410749477
Lê Quang Đạo: 4008/6620
0.0183410749477
Sultan: 4009/6620
-0.539582420781
Osman II: 4010/6620
-0.539582420781
Ohrili Hüseyin Pasha: 4011/6620
-0.539582420781
Hetman: 4012/6620
0.547386487189
Jan Karol Chodkiewicz: 4013/6620
1.13146518284
Regimentarz: 4014/6620
0.547386487189
Stanisław Lubomirski (1583–1649): 4015/6620
0.547386487189
Władysław IV Vasa: 4016/6620
0.547386487189
Hetmans of Ukrainian Cossacks: 4017/6620
0.547386487189
Petro Konashevych-Sahaidachny: 4018/6620
0.547386487189
John III Sobieski: 4019/6620
1.0264166968
Michał Kazimierz Radziwiłł: 4020/6620
0.5
Grigore I Ghica: 4021/6620
0.5
Semyon Budyonny: 4022/6620
-1.06652306116
Semyon Timoshenko: 4023/6620
-1.09448032746
Heinz Guderian: 4024/6620
0.545783143715
Alfred Sully: 4025/6620
0.984614415241
Gall (Native American leader): 4026/6620
-0.0221518165484
Eoin O'Duffy: 4027/6620

Shiro Makino: 4185/6620
-0.443561107164
United States Third Fleet: 4186/6620
0.413384451059
Thomas C. Kinkaid: 4187/6620
-0.0948189487546
United States Seventh Fleet: 4188/6620
0.413384451059
Clifton Sprague: 4189/6620
0.413384451059
Jesse B. Oldendorf: 4190/6620
0.413384451059
Task Force 44: 4191/6620
0.413384451059
Kiyohide Shima: 4192/6620
0.0711503637268
Jisaburō Ozawa: 4193/6620
-0.876234757153
Yukio Seki: 4194/6620
-0.405778074946
Zheng Zhilong: 4195/6620
0.440517399251
Hans Putmans: 4196/6620
-0.432780097861
Liu Xiang (pirate): 4197/6620
-0.432780097861
Li Guozhu: 4198/6620
-0.432780097861
Aleksey Kuropatkin: 4199/6620
-1.55152304627
Ōyama Iwao: 4200/6620
1.58293090756
David Rice Atchison: 4201/6620
-0.5
Gérard Leman: 4202/6620
-0.499408259775
Otto von Emmich: 4203/6620
0.507271455603
William Hoste: 4204/6620
0.570661173332
Bernard Dubourdieu: 4205/6620
-0.562937989518
Carlo Pellion di Persano: 4206/6620
-0.496068226645
Crazy Horse: 4207/6620
0.488815171223
Lame White Man: 4208/

-0.581207934498
Humphrey Marshall (general): 4360/6620
-0.505071756934
James A. Garfield: 4361/6620
0.512932754419
Olaf Helset: 4362/6620
0.503931773355
Oliver Møystad: 4363/6620
0.503931773355
Eberhard Spiller: 4364/6620
-0.496068226645
Chūichi Nagumo: 4365/6620
0.0978998103418
Samuel C. Means: 4366/6620
-0.413982884041
Gerhard Kauffmann: 4367/6620
0.414188970951
Friedrich Weber (general): 4368/6620
0.414188970951
Adrianus Antonius van Nijnatten: 4369/6620
-0.406578051555
George B. Crittenden: 4370/6620
-0.511145454024
Felix Zollicoffer: 4371/6620
-0.511145454024
Giovanni Marchese di Provera: 4372/6620
-0.413876619088
Cyril Clowes: 4373/6620
0.438078518449
Masajiro Hayashi: 4374/6620
-0.430351104015
Minoru Yano: 4375/6620
-0.430351104015
Albert G. Noble: 4376/6620
0.440158546784
Roscoe B. Woodruff: 4377/6620
0.86334770811
Clarence A. Martin: 4378/6620
0.440158546784
Wendell Fertig: 4379/6620
0.440158546784
Salipada Pendatun: 4380/6620
0.440158546784
Paulino Santos: 4381/6620
-0.056833

-0.465374651713
Barthélemy Catherine Joubert: 4540/6620
0.0460462509282
Sule Apollo: 4541/6620
0.5
Martin Adamu: 4542/6620
0.5
H.M. Njoku: 4543/6620
-0.5
Benjamin Huger (general): 4544/6620
-0.5
Samuel P. Heintzelman: 4545/6620
0.5
George MacDonnell: 4546/6620
0.467669554122
Benjamin Forsyth: 4547/6620
-0.459847323479
Simon Bolivar Buckner Jr.: 4548/6620
0.847521782145
Joseph Stilwell: 4549/6620
0.434691291386
Bernard Rawlings (Royal Navy officer): 4550/6620
0.434691291386
Mitsuru Ushijima: 4551/6620
-0.426978225307
Isamu Chō: 4552/6620
-0.426978225307
Hiromichi Yahara: 4553/6620
-0.426978225307
Minoru Ōta: 4554/6620
-0.426978225307
Seiichi Itō: 4555/6620
-0.426978225307
Matthew Butler: 4556/6620
-0.5
Alfred Thomas Archimedes Torbert: 4557/6620
0.974370471022
Arend Dickmann: 4558/6620
0.5
Grigoriy Vladimirovich Rosen: 4559/6620
-0.0501088060984
Józef Chłopicki: 4560/6620
0.0578806447116
Jan Zygmunt Skrzynecki: 4561/6620
0.0578806447116
Jan Krukowiecki: 4562/6620
-0.398488367055
Peter A

In [ ]:
df_totalwar = pd.DataFrame(war_list).sort_values('WAR', ascending = False).reset_index(drop = True)

In [ ]:
df_totalwar[df_totalwar.General == 'Pyrrhus of Epirus']

In [ ]:
df_run = df_run.dropna(subset = ['belligerent'])

In [ ]:
df_totalwar = df_totalwar[df_totalwar.General != 'Capital punishment']
df_totalwar = df_totalwar[df_totalwar.General != 'Lieutenant general']

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(df_totalwar['WAR'], kde = False, bins = 30)
plt.title("Distribution of Generals' War")
plt.ylabel('Number of Generals')
plt.annotate('Napoleon', xy = (16.713, 1), xytext = (17.713, 400), arrowprops = dict(facecolor = 'black'))
plt.annotate('Julius Caesar', xy = (7.364545, 1), xytext = (8.364545, 400), arrowprops = dict(facecolor = 'black'))
plt.tight_layout()
plt.savefig('/home/ethanarsht/Downloads/distplot.png')

In [ ]:
def index():
    output_file('docs/index.html')
    source = ColumnDataSource(data = dict(battles = df_totalwar.Battles,
                                          war = df_totalwar.WAR,
                                          general = df_totalwar.General))

    hover = HoverTool(tooltips = [('General', '@general'),
                                  ('Battles', '@battles'),
                                  ('WAR', '@war'),
                                      ])
    p = figure(title = "Click a circle for a profile of that general's career WAR",
               plot_height = 600, 
               plot_width = 800,

               tools = [hover, "tap", 'box_zoom', ResetTool()])
    p.circle(x = 'war', y = 'battles', size = 10, source = source)
    p.xaxis.axis_label = 'WAR'
    p.yaxis.axis_label = 'Battles'

    link = "{}.html".format('@general')

    if link == 'https://ethanarsht.github.io/military_rankings/Thomas de Courtenay, 5th/13th Earl of Devon':
            link = 'https://ethanarsht.github.io/military_rankings/Thomas de Courtenay'
    url = link
    taptool = p.select(type=TapTool)
    taptool.callback = OpenURL(url=url)
    save(p)

In [ ]:
index()

In [78]:
def bokeh(commander):
    global df_battles
    
    df_battles = pd.DataFrame()
    commander_war(commander)
    
#     df_battles = df_battles[df_battles.General == commander]
    df_battles.Year = df_battles.Year.fillna(-5000)
    df_battles.Year = df_battles.Year.astype(float, errors = 'ignore')
    df_battles = df_battles.sort_values('Year')
    wl = pd.DataFrame(df_battles.Outcome.value_counts())
    
    if 'V' in wl.index:
        wins = wl.loc['V', 'Outcome']
    else:
        wins = 0
    if 'D' in wl.index:
        losses = wl.loc['D', 'Outcome']
    else:
        losses = 0
    if 'I' in wl.index:
        draws = wl.loc['I', 'Outcome']
    else:
        draws = 0
    df_battles = df_battles.reset_index(drop = True)
    for i, row in df_battles.iterrows():
        df_battles.ix[i, 'cumulative'] = sum(df_battles.Value[0:i+1])
    
    
    source = ColumnDataSource(data = dict(battle = df_battles.Battle,
                              general = df_battles.General,
                              outcome = df_battles.Outcome,
                              value = df_battles.Value,
                              year = df_battles.Year,
                              cumulative = df_battles.cumulative))
    if all(x != -5000 for x in df_battles.Year):
        hover = HoverTool(tooltips = [('Battle', '@battle'),
                                  ('Outcome', '@outcome'),
                                  ('WAR', '@value'),
                                  ('Cumulative WAR', '@cumulative'),
                                  ('Year', '@year')])
    else:
        hover = HoverTool(tooltips = [('Battle', '@battle'),
                                  ('Outcome', '@outcome'),
                                  ('WAR', '@value'),
                                  ('Cumulative WAR', '@cumulative')
                                  ])
    
    x = list(df_battles.Battle)
    y = list(df_battles.Value)
    y_cum = list(df_battles.cumulative)
    
    if commander == 'Thomas de Courtenay, 5th/13th Earl of Devon':
        commander = 'Thomas de Courtenay'
    
    output_file('docs/{}.html'.format(commander))
    
    p = figure(title = commander + ' ({} - {} - {})'.format(wins, draws, losses), 
               plot_width=800, 
               plot_height=600, 
               x_range = x, 
               tools = [hover, 'tap']
               )
    p.xaxis.axis_label = "Click on a bar for more info on the battle it represents"
    p.yaxis.axis_label = "Cumulative WAR"
    
    url = 'https://en.wikipedia.org/wiki/{}'.format('@battle')
    taptool = p.select(type=TapTool)
    taptool.callback = OpenURL(url=url)

    p.line(x = 'battle', y = 'cumulative', source = source, line_width = 5)
#     g1_r = p.add_glyph(source_or_glyph = source, glyph = g_2)
#     g1_tap = TapTool(renderers = [g1_r],
#                     callback = OpenURL(url=url))
#     p.add_tools(g1_tap)
    
    color_mapper = CategoricalColorMapper(factors=['V', 'D', 'I'], palette=['Blue', 'Red', 'Yellow'])
    p.vbar(x = 'battle', 
           top = 'value', 
           width = 0.9, 
           color={'field': 'outcome', 'transform': color_mapper},
           alpha = 0.5,
           source = source)
    
#     url_2 = 'en.wikipedia.org/wiki/@battle'
#     g2_r = p.add_glyph(source_or_glyph = source, glyph = g_2)
#     g2_tap = TapTool(renderers = [g2_r],
#                      callback = OpenURL(url=url_2))
#     p.add_tools(g2_tap)
    
    
    p.xaxis.major_label_orientation = math.pi/2
    

#     show(p)
    save(p)

In [ ]:
count = 0
for general in df_run.belligerent.unique()[1:]:
    battle_list = []
    count += 1
    print(str(count) + '/' + str(len(df_run.belligerent.unique())))
    print(general)
    bokeh(general)

1/6620
David Ben-Gurion
0.976471504927


/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


2/6620
Yisrael Galili
0.466697501186
3/6620
Yaakov Dori
0.466697501186
4/6620
Yigael Yadin
0.466697501186
5/6620
Mickey Marcus
0.466697501186
6/6620
Yigal Allon
0.466697501186
7/6620
Yitzhak Rabin
1.0317652423
8/6620
David Shaltiel
0.466697501186
9/6620
Moshe Dayan
2.09708607981
10/6620
Shimon Avidan
0.466697501186
11/6620
Moshe Carmel
0.466697501186
12/6620
Yitzhak Sadeh
0.466697501186
13/6620
Abdul Rahman Hassan Azzam
-0.464374724019
14/6620
Farouk of Egypt
-0.464374724019
15/6620
Abdullah I of Jordan
-0.464374724019
16/6620
Muzahim al-Pachachi
-0.464374724019
17/6620
Husni al-Za'im
-0.464374724019
18/6620
Amin al-Husseini
-0.464374724019
19/6620
Ahmed Ali al-Mwawi
-0.464374724019
20/6620
Mohammed Naguib
-0.464374724019
21/6620
John Bagot Glubb
-0.464374724019
22/6620
Habis al-Majali
-0.464374724019
23/6620
Hasan Salama
-0.464374724019
24/6620
Arab Liberation Army
-0.464374724019
25/6620
Fawzi al-Qawuqji
-0.464374724019
26/6620
Menachem Begin
-0.0668211389422
27/6620
Ariel Sharon
2.1

213/6620
Asakura Yoshikage
-1.36901350668
214/6620
Timur
0.475843212782
215/6620
Shahrukh Mirza
0.475843212782
216/6620
Khalil Sultan
0.475843212782
217/6620
Miran Shah
0.475843212782
218/6620
Abu Bakr (Timurid)
0.475843212782
219/6620
Sultan Huseyn
0.475843212782
220/6620
Pir Muhammad ibn Jahangir
0.475843212782
221/6620
Taj al-Din Shah-i Shahan Abu'l Fath
0.475843212782
222/6620
Bayezid I
-0.0132332338126
223/6620
Süleyman Çelebi
-0.473515332634
224/6620
Mehmed I
-0.473515332634
225/6620
Stefan Lazarević
0.0093628799067
226/6620
Gannys
0.5
227/6620
Heraclius
-0.675561800645
228/6620
Shahrbaraz
0.425106073641
229/6620
Shahin Vahmanzadegan
0.5
230/6620
Eslamu
-0.581789644599
231/6620
Fatagar
-0.581789644599
232/6620
Theophilos (emperor)
-0.51235325206
233/6620
Theophobos
-0.51235325206
234/6620
Manuel the Armenian
-0.51235325206
235/6620
Khaydhar ibn Kawus al-Afshin
0.514685411299
236/6620
Umar al-Aqta
0.514685411299
237/6620
Jaysh ibn Samsama
0.5
238/6620
Damian Dalassenos
-0.5
239/66

0.445559129486
420/6620
List of Marshals of France
0.445559129486
421/6620
Thomas of Lancaster, 1st Duke of Clarence
-0.443254116529
422/6620
Thomas Beaufort, Duke of Exeter
-0.443254116529
423/6620
John Beaufort, 1st Duke of Somerset
-0.443254116529
424/6620
Zhang Fei
1.05280629814
425/6620
Zhang He
-0.0227512644604
426/6620
Baldwin IV of Jerusalem
0.0493866009899
427/6620
Baldwin of Ibelin
0
428/6620
Balian of Ibelin
-0.96928814934
429/6620
Hugh II of Saint Omer
0
430/6620
Gökböri
0.889375724231
431/6620
Charles I of Naples
0.5
432/6620
Manfred, King of Sicily
-0.5
433/6620
Quintus Fulvius Flaccus (consul 237 BC)
-0.0463338901092
434/6620
Hanno the Elder
-0.979641057984
435/6620
Tiberius Sempronius Gracchus (consul 215 BC)
0.501739712627
436/6620
Manius Curius Dentatus
0.5
437/6620
John II Komnenos
0.5
438/6620
Telerig of Bulgaria
-0.422826178123
439/6620
Seron (Syrian governor)
-0.560736088276
440/6620
Judas Maccabeus
2.3227287845
441/6620
Cestius Gallus
-0.5
442/6620
Simon bar Gior

...

0.568997056861
4950/6620
Henry Johnson (American Revolutionary War)
-0.464335719613
4951/6620
Carl Gustaf Armfeldt
-1.01877659013
4952/6620
William Forbes Gatacre
-0.5
4953/6620
Avraham Adan
0.592024894749
4954/6620
Yossi Yoffe
0.5
4955/6620
Nahum Zaken
0.5
4956/6620
John J. Peck
0.00957176890499
4957/6620
Harald Öhquist
0.5
4958/6620
Carl Gustaf Emil Mannerheim
0.0191032769072
4959/6620
Hector Waller
-0.397746197909
4960/6620
Albert H. Rooks
-0.397746197909
4961/6620
Kenzaburo Hara (admiral)
0.399984577858
4962/6620
Alexei Vinogradov
-0.563391734576
4963/6620
Hjalmar Siilasvuo
1.06568671477
4964/6620
Aubertin Walter Sothern Mallaby
0.480442110075
4965/6620
Robert Mansergh
0.480442110075
4966/6620
Sutomo
-0.478112253561
4967/6620
Cornelis Speelman
0.5
4968/6620
Sir Charles Green, 1st Baronet
0.5
4969/6620
Van Imbyze van Batenburg
-0.5
4970/6620
Cadmus M. Wilcox
-0.53334152834
4971/6620
Hendrik van Haus
-0.498833067244
4972/6620
João Fernandes Vieira
0.501166932756
4973/6620
Han Fuju
0.

5144/6620
Guy Simonds
0.521887746439
5145/6620
Bertram Ramsay
0.987367417797
5146/6620
Francisco de Moncada, 3rd Marquis of Aitona
-0.543280372676
5147/6620
John VIII, Count of Nassau-Siegen
-0.543280372676
5148/6620
Marinus Hollare
0.545595802943
5149/6620
Emir Abdelkader
-0.5
5150/6620
Henri d'Orléans, Duke of Aumale
0.5
5151/6620
Percy W. Nelles
0.406565235672
5152/6620
Leonard W. Murray
0.406565235672
5153/6620
Karl Dönitz
0.0956851555639
5154/6620
William Henry Harrison
1.85553784049
5155/6620
Jeffery Amherst, 1st Baron Amherst
0.809566653523
5156/6620
Patrick Edward Connor
1.02188774644
5157/6620
Jim Bridger
0.5
5158/6620
Frank North
0.5
5159/6620
Rapp Brush
0.49579134311
5160/6620
William Howard Arnold
0.49579134311
5161/6620
James M. Cushing
0.49579134311
5162/6620
Macario Peralta, Jr.
0.49579134311
5163/6620
Takeo Manjome
-0.493457747353
5164/6620
Remus von Woyrsch
-0.484027923544
5165/6620
Nils von Schoultz
-0.5
5166/6620
Henry Dundas, 3rd Viscount Melville
0.5
5167/6620
Wilg

5322/6620
Tamon Yamaguchi
0.426565906712
5323/6620
Winfield S. Cunningham
-0.424283960095
5324/6620
James Devereux
-0.424283960095
5325/6620
Paul A. Putnam
-0.424283960095
5326/6620
Henry T. Elrod
-0.424283960095
5327/6620
Jan Stanisław Sapieha
-0.53334152834
5328/6620
Thomas Arthur, comte de Lally
-0.520980754516
5329/6620
Louis Nicolas Victor de Félix d'Ollières
-0.535491167502
5330/6620
Carl Nieroth
0.51744189331
5331/6620
Otto Arnold von Paykull
-0.515110500911
5332/6620
Ivan Paskevich
0.974066977263
5333/6620
Karl Wilhelm von Toll
0.461383425807
5334/6620
Leon Trotsky
-0.500560965622
5335/6620
Hayk Bzhishkyan
-0.500560965622
5336/6620
Sergey Kamenev
-0.500560965622
5337/6620
Alexander Yegorov (military)
-0.500560965622
5338/6620
Tadeusz Jordan-Rozwadowski
0.502894803261
5339/6620
Bolesław Roja
0.502894803261
5340/6620
Leonard Skierski
0.502894803261
5341/6620
Georg von Küchler
-0.00250552221893
5342/6620
Werner von Fritsch
0.483489407276
5343/6620
Walerian Czuma
-0.481158458583
53

5511/6620
John Maxwell (British Army officer)
0.867314810691
5512/6620
William Lowe (British Army officer)
0.412601618327
5513/6620
Patrick Pearse
-0.410340918357
5514/6620
James Connolly
-0.410340918357
5515/6620
Tom Clarke (Irish republican)
-0.410340918357
5516/6620
Seán Mac Diarmada
-0.410340918357
5517/6620
Joseph Plunkett
-0.410340918357
5518/6620
Éamonn Ceannt
-0.410340918357
5519/6620
Thomas MacDonagh
-0.410340918357
5520/6620
Frederick Stanley Maude
0.916927047155
5521/6620
Halil Kut
0.0479366831523
5522/6620
Alexander Madiebo
-0.5
5523/6620
Walter Sinclair Delamain
0.5
5524/6620
Omar Hadid
0.476160255571
5525/6620
Abu Anas al-Shami
0.476160255571
5526/6620
Abdullah al-Janabi
0.0343245002813
5527/6620
Abu Ayyub al-Masri
0.534324500281
5528/6620
James T. Conway
-0.473832226509
5529/6620
James Mattis
-0.0296929500751
5530/6620
Marcus Junius Brutus the Younger
-0.976143185821
5531/6620
Aulus Allienus
-0.976143185821
5532/6620
Serapion (strategos)
-0.976143185821
5533/6620
Edmund 

0.424676950651
5701/6620
Manohar Lal Chibber
0.470299102944
5702/6620
Northern Command (India)
0.470299102944
5703/6620
Premindra Singh Bhagat
0.470299102944
5704/6620
Central Command (India)
0.470299102944
5705/6620
Sagat Singh
0.470299102944
5706/6620
IV Corps (India)
0.470299102944
5707/6620
Tapishwar Narain Raina
0.470299102944
5708/6620
II Corps (India)
0.470299102944
5709/6620
Sartaj Singh (general)
0.470299102944
5710/6620
XV Corps (India)
0.424676950651
5711/6620
I Corps (India)
0.424676950651
5712/6620
Depinder Singh
0.470299102944
5713/6620
XII Corps (India)
0.470299102944
5714/6620
J. F. R. Jacob
0.470299102944
5715/6620
Chief of staff
0.470299102944
5716/6620
Om Prakash Malhotra
0.470299102944
5717/6620
Sardarilal Mathradas Nanda
0.970299102944
5718/6620
Chief of the Naval Staff (India)
0.924676950651
5719/6620
Air chief marshal
0.838974121183
5720/6620
Pratap Chandra Lal
0.470299102944
5721/6620
Chief of the Air Staff (India)
0.424676950651
5722/6620
R. N. Kao
0.4702991029

In [ ]:
mka = df_run.loc[8965, 'belligerent']

In [ ]:
mka

In [ ]:
df_battles

In [ ]:
battle_list = []
bokeh(u'Ariel Sharon')

In [ ]:
df_warchart = pd.DataFrame({'Labels': ['Win = 1', 'Replacement-Level Performance = .51', 'WAR From Battle = .49'],
              'Values': [1, .51, .49]})

In [ ]:
df_warchart = df_warchart.sort_values('Values', ascending = True)

In [ ]:
df_warchart

In [ ]:
df_warchart = df_warchart.iloc[::-1]

In [ ]:
fig = plt.figure(figsize = (12,9))
plt.style.use('fivethirtyeight')
plt.bar(df_warchart.index, df_warchart.Values, width = 1, color = ['red', 'blue', 'yellow'])
plt.xticks(df_warchart.index + .5, df_warchart.Labels, rotation = 45)
plt.xticks()
plt.title("Bonaparte's WAR in Battle of Borodino")
plt.tight_layout()
plt.savefig('napoleon_war_sample')

# ax.set_axis_bgcolor('gray')

In [ ]:
df_warchart_two = pd.DataFrame({'Labels': ['Win = -1', 'Replacement-Level Performance = .49', 'WAR From Battle = -.51'],
              'Values': [-1, -.49, -.51]})

In [ ]:
fig = plt.figure(figsize = (12,9))
plt.style.use('fivethirtyeight')
plt.bar(df_warchart_two.index, df_warchart_two.Values, width = 1, color = ['red', 'blue', 'white'])
plt.xticks(df_warchart_two.index, df_warchart_two.Labels, rotation = 45)
plt.title("Kutuzov's WAR in Battle of Borodino")
plt.tight_layout()
plt.savefig('/home/ethanarsht/Downloads/kutuzov_war_sample')

# ax.set_axis_bgcolor('gray')